# Packages

In [2]:
import geopandas as gpd
import os
import pandas as pd
from pathlib import Path


# Data

In [3]:

# Define file path
file1_path = r"G:/Shared drives/Wellcome Trust Project Data/0_source_data/GiGL land use data/GiGL_Trees_font_point/GiGL_GLATrees_Pre2023.shp"
file2_path = r"G:/Shared drives/Wellcome Trust Project Data/0_source_data/GiGL land use data/GiGL_Trees_font_point/GiGL_GLATrees_2023-24.shp"

# Define output path
current_path = os.getcwd()
print("Current working directory:\n", current_path)

parent_folder = os.path.dirname(current_path)
print("Parent folder:\n", parent_folder)


from pathlib import Path

# Where to save
out_dir = Path(parent_folder) / "data"
out_dir.mkdir(parents=True, exist_ok=True)

# # Read the spatial data one by one
# d = gpd.read_file(file1_path)


Current working directory:
 d:\natcap\urban-cooling-health\code
Parent folder:
 d:\natcap\urban-cooling-health


## Combine tree data 

In [ ]:
# import geopandas as gpd
# import pandas as pd
# from pathlib import Path

# # --- inputs ---
# paths = [file1_path, file2_path]  # str or Path
# out_path = Path(output_path)         # e.g., "merged.gpkg"
# out_layer = out_path.stem         # gpkg layer name

# # ---- read all, tag source ----
# gdfs = []
# for p in paths:
#     p = Path(p)
#     gdf = gpd.read_file(p, engine="pyogrio")
#     # ensure geometry column exists
#     if "geometry" not in gdf.columns:
#         raise ValueError(f"{p} has no geometry column.")
#     gdf["source_file"] = p.name
#     gdfs.append(gdf)

# if not gdfs:
#     raise ValueError("No input files read.")

# # ---- choose a target CRS ----
# # prefer the first non-None CRS; you can also set a known CRS explicitly
# target_crs = next((g.crs for g in gdfs if g.crs is not None), None)

# # ---- reproject (only when safe) ----
# aligned = []
# for g in gdfs:
#     if target_crs is None:
#         # leave CRS as-is; you may want to set one: g.set_crs("EPSG:5070", inplace=True)
#         aligned.append(g)
#     elif g.crs is None:
#         # if one file lacks CRS but you *know* it matches target_crs, set it:
#         g = g.set_crs(target_crs, allow_override=False)
#         aligned.append(g)
#     elif g.crs != target_crs:
#         aligned.append(g.to_crs(target_crs))
#     else:
#         aligned.append(g)

# # ---- align schemas (union of columns across all) ----
# all_cols = sorted(set().union(*[set(g.columns) for g in aligned]))
# # always ensure 'geometry' is present and last for readability
# all_cols = [c for c in all_cols if c != "geometry"] + ["geometry"]

# for i, g in enumerate(aligned):
#     # reindex may cast geometry -> object; we re-assert below
#     aligned[i] = g.reindex(columns=all_cols)
#     aligned[i] = gpd.GeoDataFrame(aligned[i], geometry="geometry", crs=target_crs)

# # ---- optional: clean geometries ----
# try:
#     from shapely import make_valid
#     for i, g in enumerate(aligned):
#         g["geometry"] = make_valid(g["geometry"])
# except Exception:
#     # fallback: simple fix for some invalids (not all cases)
#     for i, g in enumerate(aligned):
#         g["geometry"] = g.buffer(0)

# # drop empties if needed
# for i, g in enumerate(aligned):
#     aligned[i] = g[~g.geometry.is_empty & g.geometry.notnull()]

# # ---- concatenate ----
# combined = pd.concat(aligned, ignore_index=True, sort=False)
# combined = gpd.GeoDataFrame(combined, geometry="geometry", crs=target_crs)

# # ---- sanity checks (optional) ----
# # print(combined.geom_type.value_counts(dropna=False))
# # print(combined.crs)

# # ---- write (GeoPackage) ----
# combined.to_file(out_path, driver="GPKG", layer=out_layer)
# print(f"Wrote {len(combined)} features to {out_path} (layer='{out_layer}')")


## tree species at risk

In [ ]:
## Load the tree list that will be at risk 
f = r"D:\natcap\urban-cooling-health\data\tree_list_GiGL_combined_2050.csv"

at_risk = pd.read_csv(f)

## left join the full data
trees_at_risk = d.merge(at_risk[['TaxonName', 'country', 'city', 'family', 'year']], 
                        how="left", on="TaxonName")


print(at_risk.columns)
print(trees_at_risk.columns)

# Step 2: Keep only columns from df + geometry
cols_to_keep = list(at_risk.columns) + ['geometry']
gdf_trimmed = trees_at_risk[cols_to_keep]


# Step 3: Save as new shapefile
gdf_trimmed.to_file(output_path)

print(f"Saved merged shapefile to: {output_path}")


# Stats

## Load data

In [5]:
## Load all the saved data

f1 = os.path.join(parent_folder, "data", os.path.basename(file1_path).replace('.shp', '') + "_risk_2050.shp"); print(f1)
f2 = os.path.join(parent_folder, "data", os.path.basename(file2_path).replace('.shp', '') + "_risk_2050.shp"); print(f2)


# Read the spatial data
tree_risk_layer1 = gpd.read_file(f1, engine="pyogrio")
tree_risk_layer2 = gpd.read_file(f2, engine="pyogrio")


d:\natcap\urban-cooling-health\data\GiGL_GLATrees_Pre2023_risk_2050.shp
d:\natcap\urban-cooling-health\data\GiGL_GLATrees_2023-24_risk_2050.shp


In [8]:
import os
import geopandas as gpd
import pandas as pd
from pathlib import Path

# You already have:
# f1, f2
# tree_risk_layer1 = gpd.read_file(f1, engine="pyogrio")
# tree_risk_layer2 = gpd.read_file(f2, engine="pyogrio")

g1 = tree_risk_layer1.copy()
g2 = tree_risk_layer2.copy()

# Tag source
g1["source_file"] = Path(f1).name
g2["source_file"] = Path(f2).name

# Pick a target CRS (prefer the first non-None)
target_crs = g1.crs or g2.crs

# Align CRS
def _align_crs(g, target):
    if target is None:
        return g
    if g.crs is None:
        return g.set_crs(target)
    if g.crs != target:
        return g.to_crs(target)
    return g

g1 = _align_crs(g1, target_crs)
g2 = _align_crs(g2, target_crs)

# Union schema (keep geometry last)
all_cols = [c for c in sorted(set(g1.columns) | set(g2.columns)) if c != "geometry"] + ["geometry"]
g1 = gpd.GeoDataFrame(g1.reindex(columns=all_cols), geometry="geometry", crs=target_crs)
g2 = gpd.GeoDataFrame(g2.reindex(columns=all_cols), geometry="geometry", crs=target_crs)

# Fix invalid geometries (best effort)
try:
    from shapely import make_valid
    g1["geometry"] = make_valid(g1.geometry)
    g2["geometry"] = make_valid(g2.geometry)
except Exception:
    g1["geometry"] = g1.buffer(0)
    g2["geometry"] = g2.buffer(0)

# Drop empty/null geoms
g1 = g1[g1.geometry.notnull() & ~g1.geometry.is_empty]
g2 = g2[g2.geometry.notnull() & ~g2.geometry.is_empty]

# Concatenate
combined = gpd.GeoDataFrame(
    pd.concat([g1, g2], ignore_index=True, sort=False),
    geometry="geometry",
    crs=target_crs
)

# (Optional) drop exact duplicates (including geometry)
# combined = combined.drop_duplicates(subset=[c for c in combined.columns if c != "geometry"] + ["geometry"])

# Save
out_path = os.path.join(parent_folder, "data", "GiGL_GLATrees_at_risk_2050_merged.gpkg")
layer_name = Path(out_path).stem  # "GiGL_GLATrees_at_risk_2050_merged"

combined.to_file(out_path, driver="GPKG", layer=layer_name)
print(f"Wrote {len(combined)} features to {out_path} (layer='{layer_name}')")


Wrote 1050054 features to d:\natcap\urban-cooling-health\data\GiGL_GLATrees_at_risk_2050_merged.gpkg (layer='GiGL_GLATrees_at_risk_2050_merged')


In [9]:
import pandas as pd

# --- make 'year' sortable & consistent ---
if 'year' not in combined.columns:
    raise KeyError("'year' column not found in combined")

# if it's datetime-like, grab the year; else coerce to numeric
if pd.api.types.is_datetime64_any_dtype(combined['year']):
    combined['year'] = combined['year'].dt.year.astype('Int64')
else:
    combined['year'] = pd.to_numeric(combined['year'], errors='coerce').astype('Int64')

# --- summarize ---
summary = (
    combined
    .groupby('year', dropna=False)
    .size()
    .rename('count')
    .reset_index()
    .sort_values('year', na_position='last')
)

total_n = len(combined)
summary['percent'] = (summary['count'] / total_n * 100).round(2)

# for display, keep a nice label while preserving numeric 'year'
summary['year_label'] = summary['year'].astype(str).replace('<NA>', 'Unknown')

print(summary[['year', 'year_label', 'count', 'percent']])


   year year_label   count  percent
0  2050       2050  657622    62.63
1  <NA>    Unknown  392432    37.37


## 10m grid

Turn the 1 m × 1 m grid to a 10 m × 10 m grid -- snap cells to a 10 m lattice and aggregate. This avoids creating 100 overlapping squares per tile.

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box

gdf = combined.copy()

# 0) Ensure CRS is metric (meters)
if gdf.crs is None or not gdf.crs.is_projected:
    raise ValueError("Project to a metric CRS first (e.g., EPSG:5070 or UTM).")

# 1) Define 10 m grid origin (snap to a multiple of 10 m)
cell = 10.0
xmin, ymin, xmax, ymax = gdf.total_bounds
x0 = np.floor(xmin / cell) * cell
y0 = np.floor(ymin / cell) * cell

# 2) Compute grid indices for each 1 m cell using its lower-left corner
b = gdf.geometry.bounds  # DataFrame with minx, miny, maxx, maxy
i = np.floor((b["minx"] - x0) / cell).astype("int64")
j = np.floor((b["miny"] - y0) / cell).astype("int64")
gdf["i"] = i
gdf["j"] = j

# 3) Build unique 10 m tiles as polygons
keys = gdf[["i", "j"]].drop_duplicates().reset_index(drop=True)
xs = x0 + keys["i"].to_numpy() * cell
ys = y0 + keys["j"].to_numpy() * cell
geoms = [box(x, y, x + cell, y + cell) for x, y in zip(xs, ys)]
tiles10 = gpd.GeoDataFrame(keys, geometry=geoms, crs=gdf.crs)

# 4) (Optional) carry aggregations to 10 m grid
# Count how many 1 m cells fell into each 10 m tile (0–100, typically <=100 if partial coverage)
counts = gdf.groupby(["i", "j"]).size().rename("n_cells").reset_index()
tiles10 = tiles10.merge(counts, on=["i", "j"], how="left")

# Example: majority year per 10 m tile (if you have a 'year' column)
if "year" in gdf.columns:
    per_year = gdf.groupby(["i", "j", "year"]).size().rename("n").reset_index()
    idx = per_year.groupby(["i", "j"])["n"].idxmax()
    majority_year = per_year.loc[idx, ["i", "j", "year"]]
    tiles10 = tiles10.merge(majority_year, on=["i", "j"], how="left")





# --- Best option: GeoPackage ---
gpkg_path = out_dir / "GiGL_GLATrees_at_risk_2050_merged_tiles10.gpkg"
layer_name = "tiles10"

# keep grid keys as ints
t10 = tiles10.copy()
for c in ("i", "j"):
    if c in t10.columns:
        t10[c] = t10[c].astype("int64")

t10.to_file(gpkg_path, driver="GPKG", layer=layer_name)  # engine="pyogrio" also OK
print(f"Wrote {len(t10)} tiles to {gpkg_path} (layer='{layer_name}')")

# --- (Optional) Shapefile (use short field names, 32-bit ints) ---
# shp_path = out_dir / "tiles10_shp" / "tiles10.shp"
# (out_dir / "tiles10_shp").mkdir(exist_ok=True)
# t_shp = t10.copy()
# for c in ("i", "j"):
#     if c in t_shp.columns:
#         t_shp[c] = t_shp[c].astype("int32")
# t_shp.to_file(shp_path, driver="ESRI Shapefile")
# print(f"Also wrote Shapefile to {shp_path}")


Wrote 777007 tiles to d:\natcap\urban-cooling-health\data\tiles10.gpkg (layer='tiles10')


## generate lc scenario - tree at risk

only changes LC pixels that (a) overlap the risk polygons and (b) are currently one of your tree codes:

In [4]:
import os
from pathlib import Path

import numpy as np
import geopandas as gpd
import rasterio
from rasterio.features import rasterize


## Load shapefile with tree cover will be at risk or loss
shp_tree_risk = out_dir / "GiGL_GLATrees_at_risk_2050_merged_tiles10.gpkg"
tree_cover_at_risk = gpd.read_file(shp_tree_risk)

## Filter the tree cover data with year = 2050, which indicate these trees will be at risk by 2050
tree_cover_at_risk = tree_cover_at_risk[tree_cover_at_risk['year'].isin([2050])]

## land cover data
lc_input = r"G:\Shared drives\Wellcome Trust Project Data\1_preprocess\UrbanCoolingModel\EP_preliminary_tests\clipped_lulc\UKECH\LCM2021_london.tif"
lc_output = lc_input.replace(".tif", "_scenario3_TreeRisk.tif")
tree_cover_code = [1, 2]
after_risk_lc_code = 5  # 5: 'Neutral Grassland',



# Ensure output folder exists
Path(os.path.dirname(lc_output)).mkdir(parents=True, exist_ok=True)

# --- Load and filter polygons ---
tree_cover_at_risk = gpd.read_file(shp_tree_risk)
tree_cover_at_risk = tree_cover_at_risk[tree_cover_at_risk["year"].isin([2050])]
tree_cover_at_risk = tree_cover_at_risk[
    tree_cover_at_risk.geometry.notna() & ~tree_cover_at_risk.geometry.is_empty
]

# --- Read raster ---
with rasterio.open(lc_input) as src:
    meta      = src.meta.copy()
    transform = src.transform
    lc_crs    = src.crs
    nodata    = src.nodata
    arr       = src.read(1)  # 2D array of land cover codes

# Reproject polygons to raster CRS if needed
if tree_cover_at_risk.crs and (tree_cover_at_risk.crs != lc_crs):
    tree_cover_at_risk = tree_cover_at_risk.to_crs(lc_crs)


# --- Rasterize risk polygons to a mask (1 = at risk) ---
shape_mask = rasterize(
    [(geom, 1) for geom in tree_cover_at_risk.geometry],
    out_shape=arr.shape,
    transform=transform,
    fill=0,
    dtype="uint8",
    # all_touched=True,  # uncomment if you want a slightly more inclusive burn-in
)

# --- Build masks ---
valid_mask = np.ones_like(arr, dtype=bool)
if nodata is not None:
    valid_mask &= (arr != nodata)

# If LC==0 is background you want to preserve, keep it out of edits:
valid_mask &= (arr != 0)

# Only modify cells that are both "at risk" and currently a tree class
tree_mask = np.isin(arr, tree_cover_code)
target_mask = (shape_mask == 1) & tree_mask & valid_mask

# --- Apply scenario ---
remapped = arr.copy()
remapped[target_mask] = after_risk_lc_code

# --- Save updated raster ---
meta_out = meta.copy()
# Keep original dtype unless you explicitly want uint8
meta_out.update(
    dtype=remapped.dtype,
    compress="lzw",
    nodata=nodata
)

with rasterio.open(lc_output, "w", **meta_out) as dst:
    dst.write(remapped, 1)

print(f"Updated land cover raster saved at: {lc_output}")


Updated land cover raster saved at: G:\Shared drives\Wellcome Trust Project Data\1_preprocess\UrbanCoolingModel\EP_preliminary_tests\clipped_lulc\UKECH\LCM2021_london_scenario3_TreeRisk.tif
